# This file is created to Change the baseline dataset into a dupedataset 

In [1]:
import os
import pandas as pd

Address_df = pd.read_csv('Data_source/Sample_35_train/train_data/Address.csv')  
ContactPoint_df = pd.read_csv('Data_source/Sample_35_train/train_data/ContactPoint.csv')
HealthcareOrganization_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcareOrganization.csv')
HealthcarePersonnel_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcarePersonnel.csv')
Person_df = pd.read_csv('Data_source/Sample_35_train/train_data/Person.csv')
ServiceDepartment_df = pd.read_csv('Data_source/Sample_35_train/train_data/ServiceDepartment.csv')

# Specify the encoding to handle the decoding error
golden_record = pd.read_csv('train_golden_standard_duplicates.csv', encoding='latin1')

In [2]:
golden_record

,original_id,duplicate_id,entity_type,variation_type,field_name,original_value,varied_value
0,1436d260-d434-492d-8bbe-ea054653ff61,733f2829-b367-40d0-9d54-0255fb1d00f9,Address,country_expansion,country,NL,Netherlands
1,514f652f-cb71-4e2f-bbe1-4cbd9b7f3e1b,4ff53a46-c83b-41ec-bf4f-08cbf50a2283,Address,country_expansion,country,AT,Austria
2,ee7150cd-f25b-4180-a41f-26d3b6f6fec4,4623e088-eb6e-43fa-a628-e02a0fcfe1d2,Address,city_typo,city,Sleen,Sleeen
3,b4652600-92c4-4c99-9062-5b3bd2385cf5,b2b1e266-1fa9-4f81-b700-d92eb5069c98,Address,house_number_suffix,text,Sarap Crest 87,Sarap Crest 87A
4,10c2d392-a9db-434c-9322-21e2fcb0959b,9e66f283-16e3-45e1-a732-2c671315edc5,Address,country_expansion,country,EE,Estonia
...,...,...,...,...,...,...,...
2223,af2b8e1b-fe5f-4e3b-9c19-85fceda86200,8f7982fa-fb05-466d-949a-36f6dcc1f3c5,ContactPoint,email_typo,email,PriceRobinson.Musculoskeletal@dept.healthcare.org,PriceRobinson.Muscuolskeletal@dept.healthcare.org
2224,d57a00a3-4600-4654-b637-9ee556bf4493,92146f4e-18fe-4980-8f86-4214c798a381,ContactPoint,email_domain_change,email,Baker.Cardiovascular@dept.healthcare.org,Baker.Cardiovascular@dept.healthcare.de
2225,68159dd7-e780-4601-9430-efd12d92532d,5621c94c-5df4-4f49-8d72-8ddda4b2cdc9,ContactPoint,email_domain_change,email,WilliamsWilliams.Dentistry@dept.healthcare.org,WilliamsWilliams.Dentistry@dept.healthcare.nl
2226,abbf66c7-680a-4e2a-866b-d06fa2b4d215,bbd0f8d6-e3a3-4e29-8576-8d2c44650805,ContactPoint,email_typo,email,LongMartinez@healthcare.org,LongzMartinez@healthcare.org


In [3]:
# Dictionary to store all dataframes by entity type
entity_dataframes = {}

# Get a static list of variable names first to avoid the iteration error
variable_names = list(locals().keys())

# Populate the dictionary with your existing dataframes
for var_name in variable_names:
    if var_name.endswith('_df'):
        entity_type = var_name.replace('_df', '')
        entity_dataframes[entity_type] = locals()[var_name]

# Iterate through each row in the golden record
for _, row in golden_record.iterrows():
    entity_type = row['entity_type']
    
    # Check if we have a dataframe for this entity type
    if entity_type in entity_dataframes:
        df = entity_dataframes[entity_type]
        
        # Find the matching row using original_id
        mask = df['identifier'] == row['original_id']
        
        # If a match is found, update the specified field with the varied value
        if mask.any():
            field_name = row['field_name']
            if field_name in df.columns:
                df.loc[mask, field_name] = row['varied_value']
                print(f"Updated {entity_type} - {row['original_id']} - {field_name}: {row['varied_value']}")
            else:
                print(f"Warning: Field {field_name} not found in {entity_type} dataframe")
        else:
            print(f"Warning: No row with identifier {row['original_id']} found in {entity_type} dataframe")
    else:
        print(f"Warning: No dataframe found for entity type {entity_type}")

Updated Address - 1436d260-d434-492d-8bbe-ea054653ff61 - country: Netherlands
Updated Address - 514f652f-cb71-4e2f-bbe1-4cbd9b7f3e1b - country: Austria
Updated Address - ee7150cd-f25b-4180-a41f-26d3b6f6fec4 - city: Sleeen
Updated Address - b4652600-92c4-4c99-9062-5b3bd2385cf5 - text: Sarap Crest 87A
Updated Address - 10c2d392-a9db-434c-9322-21e2fcb0959b - country: Estonia
Updated Address - 8fdd2315-6676-44ce-bcea-20e0c497449d - text: GontÅ¡arov Square 129B
Updated Address - ec5a35f4-9cc6-4c68-be2d-31cf09525f64 - text: Marie-Jung-Ring 148C
Updated Address - 024e8609-4486-4d26-8fd0-92769c017e5d - country: Estonia
Updated Address - 6e695b3f-4a17-4d31-934c-cba77dc7b782 - text: Veronika-Himmelbauer-StraÃe 79B
Updated Address - 2d58cc88-a9ac-4915-bee9-db39b0343d89 - country: Austria
Updated Address - 1e73bbc2-9c9d-4a1e-b385-cc03177a776f - postalCode: 52268 
Updated Address - 4abaea30-7bfe-45c5-94c3-aaaaeceb437a - country: Netherlands
Updated Address - 2b274e9e-3f43-477c-b32f-5dd17cac29bd - 

In [4]:
HealthcareOrganization_df

,identifier,healthcareOrganizationName,address,contactPoint
0,71545a13-7a1d-4006-8d72-3104f77383c1,FH Zorg,c8a70639-eb11-47b3-a7a9-c3787c65c1e5,23a7711a-8133-4876-b7eb-dcd9e87a1613
1,6288e1a5-cc45-4821-98a6-416d1775336d,"Garrett, Salazar and Claytno Tervisekeskus",d3290a4c-b5d3-4b16-a619-4cb1d71037d1,b421eaeb-5340-47ca-baf3-897a3e70f16a
2,5f3f5638-3870-4a14-b490-b6081dfc8352,"Green, Jones and Shelton Tervisekeskus",9cdeb3e6-0870-415c-afcd-81b5d24bace4,09e469e6-ec62-42c8-a648-ee38e07405eb
3,870f084c-7244-4536-a85e-25b4b3969057,"Nash, Andergson and Collins Zorg",971c702d-5bf4-4c04-ac64-2b4c49b25ded,14aa451c-a69c-4b85-9432-f8db6a174c1c
4,ccc14d51-73f6-40d8-a9f4-1cc04653a560,Lopez-Key Gesundheitszentrum,0cc36d8c-7786-4fe5-9675-ebf74fe30c9a,b9bdee2d-d663-449d-955e-18b1fa83ada4
5,fb5eb866-2640-411e-a9f2-c3c74505f4f6,Williams and Sgns Gesundheitszentrum,4745dd9e-2789-4389-9f32-77fd1d77ce40,f24dfdd8-5091-4bdc-8ef0-66d44279b14d
6,f2ad985f-ff3e-4ba1-8ac7-28b4a41865bf,BI Tervisekeskus,15ace7a1-ceca-4ee3-90da-8a9516408169,8e751eb7-64d0-4913-991b-8adf0202861c
7,c2472fd6-03e9-4a02-8cea-2df00a66dc4e,GPLC Gesundheitszentrum,d3b564b0-8be0-4c3e-9c94-938160c6b3ed,a36bcb01-67e9-4363-905c-053b25fdacbe
8,f164f9d8-4312-4ce2-9c21-51e17e56ac3d,Santana-Shw Zorg,8147a8f4-5f0e-4320-b7f6-0e7f75f2bc20,ce1bb02a-cb4d-48d6-adb6-da351734a26c
9,586f1721-0785-48d7-b118-2d235bf80676,"Chan, Hawle and Fisher Zorg",e38690e7-e27a-48e9-91c3-d1bcc6be6432,6d0c62c3-254b-47ae-9d0a-b994f20b575d


In [5]:
from faker import Faker

fake = Faker()

# Create a copy to avoid modifying original data
A_copy = Address_df.copy()
CP_copy = ContactPoint_df.copy()
P_copy = Person_df.copy()
HO_copy = HealthcareOrganization_df.copy()
SD_copy = ServiceDepartment_df.copy()
HP_copy = HealthcarePersonnel_df.copy()

# Save old values
old_valuesHO = {
    'identifier': HO_copy['identifier'].copy(),
    'address': HO_copy['address'].copy(),
    'contactPoint': HO_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field
HO_copy['identifier'] = HO_copy['identifier'].apply(lambda x: fake.uuid4())
HO_copy['address'] = HO_copy['address'].apply(lambda x: fake.uuid4())
HO_copy['contactPoint'] = HO_copy['contactPoint'].apply(lambda x: fake.uuid4())

# Create copies of dataframes
old_valuesSD = {
    'identifier': SD_copy['identifier'].copy(),
    'address': SD_copy['address'].copy(),
    'contactPoint': SD_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field in ServiceDepartment_df
SD_copy['identifier'] = SD_copy['identifier'].apply(lambda x: fake.uuid4())
SD_copy['address'] = SD_copy['address'].apply(lambda x: fake.uuid4())
SD_copy['contactPoint'] = SD_copy['contactPoint'].apply(lambda x: fake.uuid4())

old_valuesHP = {
    'identifier': HP_copy['identifier'].copy()
}
# Generate new UUIDs for Healthcare Personnel identifiers
HP_copy['identifier'] = HP_copy['identifier'].apply(lambda x: fake.uuid4())

In [6]:
HO_copy

,identifier,healthcareOrganizationName,address,contactPoint
0,6d15bdd9-73d7-41e4-b7fc-c7b887f931eb,FH Zorg,4c177a82-236d-40b7-84f8-29572dab5f8d,28c50c47-b8ee-473b-8a31-e25b9e373838
1,bed1343f-3209-412f-8e77-05b84d36818c,"Garrett, Salazar and Claytno Tervisekeskus",48d0b306-a22b-4b55-9a00-25208a608e6f,e471dd9d-d348-4fdc-a76e-f4f7f201391f
2,446e14bc-8a69-41d4-b519-b2e2df5daf95,"Green, Jones and Shelton Tervisekeskus",1557608b-eedb-4d9e-a29a-0f9c5b2e9a11,97c53f76-ba5e-42d6-872a-79503b8513b2
3,bbeba19d-6f0b-417f-8437-117d02cd697e,"Nash, Andergson and Collins Zorg",4cc6970b-e19e-41d8-a7f2-3a1fc6a3ef0b,2a33b808-52ed-4ac2-b8f0-615f7d56f861
4,6e9fe384-b713-4b6a-95b6-466df1e7d14b,Lopez-Key Gesundheitszentrum,54f37d27-a279-429b-9e10-796af61538f3,d950d460-00f8-497f-8df0-eca2957361b2
5,58a6ea0b-9820-40aa-9ec5-e7ff30affa63,Williams and Sgns Gesundheitszentrum,52515e66-1745-4a36-a238-c99669087af9,2d041a84-a2e7-4370-afb8-f787e2c46df2
6,65ed857a-7d39-4b12-994a-27e4aafc021d,BI Tervisekeskus,5fb50538-36fc-40b7-b127-a1ae4c99fcd1,0007c9af-e50a-491c-8f5f-a53dcb7575a5
7,2d82b465-6161-4cf9-847c-e12e20b9cdb7,GPLC Gesundheitszentrum,874c912c-f097-4242-a5c5-f61d09f622b8,d3032047-030e-4651-b26d-ea61b06718e4
8,f2c3d0af-e219-4865-8269-93ef69921bda,Santana-Shw Zorg,160eedc2-1207-4cc9-b6c0-b9041341c829,73849c0e-d4a3-4969-93ab-6184dd55663b
9,03edf113-fca6-42fb-bf5c-57fdf62da401,"Chan, Hawle and Fisher Zorg",29ac49d9-2ff1-42ae-ac0f-4149c1101822,ec100e7b-984d-4654-897f-266ae95d28d1


In [7]:
# Create a mapping of old to new UUIDs
HO_uuid_mapping = {
    old_valuesHO['identifier'].iloc[i]: HO_copy['identifier'].iloc[i]
    for i in range(len(HO_copy))
}

# Update references in other dataframes (using the copies)
# Update institution references in Healthcare_Personnel_df copy
HP_copy['institution'] = HP_copy['institution'].map(HO_uuid_mapping).fillna(HP_copy['institution'])

# Update isPartOf references in ServiceDepartment_df copy
SD_copy['isPartOf'] = SD_copy['isPartOf'].map(HO_uuid_mapping).fillna(SD_copy['isPartOf'])

# Update contact point references
ho_contact_mapping = {
    old_valuesHO['contactPoint'].iloc[i]: HO_copy['contactPoint'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in ContactPoint_df copy where it matches old contact points
CP_copy['identifier'] = CP_copy['identifier'].map(ho_contact_mapping).fillna(CP_copy['identifier'])

# Update address references
ho_address_mapping = {
    old_valuesHO['address'].iloc[i]: HO_copy['address'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in Address_df copy where it matches old addresses
A_copy['identifier'] = A_copy['identifier'].map(ho_address_mapping).fillna(A_copy['identifier'])

# Create mapping for ServiceDepartment
sd_uuid_mapping = {
    old_valuesSD['identifier'].iloc[i]: SD_copy['identifier'].iloc[i]
    for i in range(len(SD_copy))
}

# Create address mapping for ServiceDepartment
sd_address_mapping = {
    old_valuesSD['address'].iloc[i]: SD_copy['address'].iloc[i]
    for i in range(len(SD_copy))
}

# Create contact point mapping for ServiceDepartment
sd_contact_mapping = {
    old_valuesSD['contactPoint'].iloc[i]: SD_copy['contactPoint'].iloc[i]
    for i in range(len(SD_copy))
}

# Update references in Address_df copy for ServiceDepartment
A_copy['identifier'] = A_copy['identifier'].map(sd_address_mapping).fillna(A_copy['identifier'])

# Update references in ContactPoint_df copy for ServiceDepartment
CP_copy['identifier'] = CP_copy['identifier'].map(sd_contact_mapping).fillna(CP_copy['identifier'])

# Create mapping for Personnel
hp_uuid_mapping = {
    old_valuesHP['identifier'].iloc[i]: HP_copy['identifier'].iloc[i]
    for i in range(len(HP_copy))
}

# Update references in Person_df copy
P_copy['identifier'] = P_copy['identifier'].map(hp_uuid_mapping).fillna(P_copy['identifier'])

In [8]:
old_valuesHO

{'identifier': 0     71545a13-7a1d-4006-8d72-3104f77383c1
 1     6288e1a5-cc45-4821-98a6-416d1775336d
 2     5f3f5638-3870-4a14-b490-b6081dfc8352
 3     870f084c-7244-4536-a85e-25b4b3969057
 4     ccc14d51-73f6-40d8-a9f4-1cc04653a560
 5     fb5eb866-2640-411e-a9f2-c3c74505f4f6
 6     f2ad985f-ff3e-4ba1-8ac7-28b4a41865bf
 7     c2472fd6-03e9-4a02-8cea-2df00a66dc4e
 8     f164f9d8-4312-4ce2-9c21-51e17e56ac3d
 9     586f1721-0785-48d7-b118-2d235bf80676
 10    4ec985ff-94b2-4b9d-8881-9f421a42b629
 11    788c161e-f3cc-4d8a-8b16-78e45f20f406
 12    48622a67-4a29-4067-9c66-a27b6a325333
 13    862f78e1-25c2-4b6a-b575-bc6eeee84bcb
 14    f46860a2-a604-45af-b4db-1c1eaa66b464
 15    0b5aafef-85a1-482a-8761-585b4168705a
 16    3fee2fc7-2e2d-4fdf-b57b-8a920a12f3b3
 17    1e454241-45c1-40a8-a52b-a0ce627b585f
 18    839137a6-a977-4b71-9e6d-2d7e9f312acb
 19    f1c58f44-7e08-4c1b-8da6-a326451437d6
 20    909b471c-0d1f-4a7f-90fc-e92295be263b
 21    ce8c037e-e51c-4cdb-8247-34dfdd1f8e30
 22    53f3c158-42

In [11]:
import os

# Create the new directory if it doesn't exist
if not os.path.exists('testdata_dup'):
    os.makedirs('testdata_dup')

# Save each dataframe to CSV in the new directory
A_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/Address.csv', index=False)
CP_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/ContactPoint.csv', index=False)
HO_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcareOrganization.csv', index=False)
HP_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcarePersonnel.csv', index=False)
P_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/Person.csv', index=False)
SD_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/ServiceDepartment.csv', index=False)

In [12]:
import pandas as pd
import os

# Define paths
original_data_path = 'Data_source/Sample_35_train/train_data'
duplicate_data_path = 'Data_source/Sample_35_train/train_data_dupe'
golden_standard_path = 'train_golden_standard_duplicates.csv'
output_path = 'train_golden_standard_duplicates_final.csv'

# Read golden standard file
golden_df = pd.read_csv(golden_standard_path, encoding='latin1')

# Dictionary to store matching field values between original and duplicate datasets
field_value_to_id_mapping = {}

# Process each entity type
for _, row in golden_df.iterrows():
    entity_type = row['entity_type']
    field_name = row['field_name']
    varied_value = row['varied_value']
    
    # Define file paths based on entity type
    if entity_type == 'Address':
        original_file = 'Address.csv'
        duplicate_file = 'Address.csv'
    elif entity_type == 'ContactPoint':
        original_file = 'ContactPoint.csv'
        duplicate_file = 'ContactPoint.csv'
    elif entity_type == 'HealthcareOrganization':
        original_file = 'HealthcareOrganization.csv'
        duplicate_file = 'HealthcareOrganization.csv'
    elif entity_type == 'HealthcarePersonnel':
        original_file = 'HealthcarePersonnel.csv'
        duplicate_file = 'HealthcarePersonnel.csv'
    elif entity_type == 'Person':
        original_file = 'Person.csv'
        duplicate_file = 'Person.csv'
    elif entity_type == 'ServiceDepartment':
        original_file = 'ServiceDepartment.csv'
        duplicate_file = 'ServiceDepartment.csv'
    else:
        continue
    
    # Read duplicate dataset
    duplicate_df = pd.read_csv(os.path.join(duplicate_data_path, duplicate_file))
    
    # Look for records in the duplicate dataset that have the varied value
    # in the specified field
    if field_name in duplicate_df.columns:
        matching_rows = duplicate_df[duplicate_df[field_name] == varied_value]
        
        if not matching_rows.empty:
            # Get the identifier from the first matching row
            new_duplicate_id = matching_rows.iloc[0]['identifier']
            
            # Store the mapping for this specific row
            key = (entity_type, field_name, varied_value)
            field_value_to_id_mapping[key] = new_duplicate_id

# Create a new dataframe for the updated golden standard
updated_golden_df = golden_df.copy()

# Update the duplicate_id column based on our mappings
for i, row in updated_golden_df.iterrows():
    key = (row['entity_type'], row['field_name'], row['varied_value'])
    if key in field_value_to_id_mapping:
        updated_golden_df.at[i, 'duplicate_id'] = field_value_to_id_mapping[key]

# Save the updated golden standard file
updated_golden_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"Updated golden standard file saved to {output_path}")
print(f"Total field-value mappings created: {len(field_value_to_id_mapping)}")
print(f"Duplicate IDs updated: {sum(updated_golden_df['duplicate_id'] != golden_df['duplicate_id'])}")

Updated golden standard file saved to train_golden_standard_duplicates_final.csv
Total field-value mappings created: 1910
Duplicate IDs updated: 2228
